In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.options.display.max_columns = 500

### Загрузим датасет с машинами. Цель - верно восстанавливать для каждой из них цену продажи!

In [3]:
data = pd.read_csv('autos.csv')

data.head()

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner
0,Maruti 800 AC,2007,60000,70000,Petrol,Individual,Manual,First Owner
1,Maruti Wagon R LXI Minor,2007,135000,50000,Petrol,Individual,Manual,First Owner
2,Hyundai Verna 1.6 SX,2012,600000,100000,Diesel,Individual,Manual,First Owner
3,Datsun RediGO T Option,2017,250000,46000,Petrol,Individual,Manual,First Owner
4,Honda Amaze VX i-DTEC,2014,450000,141000,Diesel,Individual,Manual,Second Owner


In [4]:
data.describe(include=object)

,name,fuel,seller_type,transmission,owner
count,4340,4340,4340,4340,4340
unique,1491,5,3,2,5
top,Maruti Swift Dzire VDI,Diesel,Individual,Manual,First Owner
freq,69,2153,3244,3892,2832


In [5]:
data.describe()

,year,selling_price,km_driven
count,4340.000000,4.340000e+03,4340.000000
mean,2013.090783,5.041273e+05,66215.777419
std,4.215344,5.785487e+05,46644.102194
min,1992.000000,2.000000e+04,1.000000
25%,2011.000000,2.087498e+05,35000.000000
50%,2014.000000,3.500000e+05,60000.000000
75%,2016.000000,6.000000e+05,90000.000000
max,2020.000000,8.900000e+06,806599.000000


In [6]:
data.shape

(4340, 8)

In [7]:
# пропущенных значений вроде бы нет
data.isna().sum()

name             0
year             0
selling_price    0
km_driven        0
fuel             0
seller_type      0
transmission     0
owner            0
dtype: int64

In [8]:
### Колонка с тергетом - "selling price"

X = data.drop("selling_price", axis=1)
y = data["selling_price"]

### Будем замерять MSLE!
### Поэтому прологарифмируем таргет
### А после оптимизируем MSE

y = y.apply(np.log1p)

In [9]:
y.head()

0    11.002117
1    11.813037
2    13.304687
3    12.429220
4    13.017005
Name: selling_price, dtype: float64

In [10]:
### Разделим выборку на трейн и тест!

from sklearn.model_selection import train_test_split 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

__Задание__ 

Реализуйте свой MeanTargetEncoder с добавленем некоторого шума!

Однажды в лекционном материале, обсуждая счетчики, мы говорили с вами о том, что из-за них модели могут переобучаться. Один из способов бороться с этим - валидировать расчеты среднего таргета (стратегия отложенной выборки / расчеты на кросс-валидации). Но есть еще проще!

Можно просто к значению счетчика добавить случайный шум (зашумить данные)!

Напомним, что рассчитываться новые признаки должны по такой формуле:

$$
g_j = \frac{\sum_{i=1}^{l} [f_j(x) = f_j(x_i)]}{l} + C * \epsilon
$$


Пусть шум будет случайной величиной из нормального стандартного распределения, то есть $\epsilon \sim N(0, 1) $, а $ C = 0.006$.

Создавай свой класс-трансформер, наследуйтесь от классов `BaseEstimator, TransformerMixin` из `sklearn.base`. Трансформер не должен модифицировать передаваемую ему выборку inplace, а все необходимые статистики нужно считать только по обучающей выборке в методе `fit`. 
Ваш трансформер должен принимать при инициализации список из категориальных признаков и список из числовых признаков. 

Если для какого-то признака в тестовой выборке отсутствует значение, трансформер должен поставить там 0.

На выходе должен получиться датасет того же размера с измененными категориальными признаками

In [11]:
object_cols = ['name', 'year', 'fuel', 'seller_type', 'transmission', 'owner']
num_cols = ['km_driven']

X.head()

,name,year,km_driven,fuel,seller_type,transmission,owner
0,Maruti 800 AC,2007,70000,Petrol,Individual,Manual,First Owner
1,Maruti Wagon R LXI Minor,2007,50000,Petrol,Individual,Manual,First Owner
2,Hyundai Verna 1.6 SX,2012,100000,Diesel,Individual,Manual,First Owner
3,Datsun RediGO T Option,2017,46000,Petrol,Individual,Manual,First Owner
4,Honda Amaze VX i-DTEC,2014,141000,Diesel,Individual,Manual,Second Owner


In [12]:
from sklearn.base import BaseEstimator, TransformerMixin

class MeanTargetEncoderNoise(BaseEstimator, TransformerMixin):
    
    def __init__(self, categorical, numeric):
        self.categorical = categorical
        self.numeric = numeric
        self.mean_target = {} # Словарь для хранения значений Mean Target для каждого категориального признака и каждого числового признака
        self.C = 0.006
    
    def fit(self, X, y):
        """
        Метод fit для расчета Mean Target для каждого категориального и числового признака.
        Расчет проводится только по обучающей выборке.
        :param X: pandas.DataFrame, обучающая выборка
        :param y: pandas.Series, значения целевой переменной для обучающей выборки
        :return: экземпляр класса MeanTargetEncoderNoise
        """
        for col in self.categorical:
            df = pd.DataFrame({'feature': X[col], 'target': y}) # Создаем DataFrame, чтобы сгруппировать значения по признаку
            mean_target = df.groupby(['feature'])['target'].mean() # Считаем среднее значение target для каждого значения признака
            self.mean_target[col] = mean_target  # Добавляем значения mean target в словарь
        
        for col in self.numeric:
            self.mean_target[col] = y.mean() # Для числовых признаков считаем среднее значение target по всей выборке
            
            return self
        
    def transform(self, X):
        """
        Метод transform для трансформации признаков с помощью Mean Target Encoding с добавлением шума.
        
        :param X: pandas.DataFrame, выборка для трансформации
        
        :return: pandas.DataFrame, трансформированная выборка
        """
        temp = X.copy()  # Создаем копию датасета, чтобы не изменять исходные данные
        
        for col in self.categorical:
            temp[col] = temp[col].map(self.mean_target[col]) # Заменяем значения категориального признака на значения mean target
            temp[col] = temp[col] + self.C * np.random.randn(len(X)) # Добавляем случайный шум
            temp[col] = temp[col].fillna(0) # Заполняем пропущенные значения 0
            
        for col in self.numeric:
            temp[col] = temp[col].fillna(self.mean_target[col]) # Заполняем пропущенные значения средним значением target по всей выборке
            
        return temp

In [93]:
### Проверка работы трансформера

np.random.seed(1)
transformer = MeanTargetEncoderNoise(categorical=object_cols, numeric=num_cols)

transformer.fit(X_train, y_train)

train = transformer.transform(X_train)
test = transformer.transform(X_test)

print(train.shape)
train.head()
#train.isna().sum()


(3472, 7)


,name,year,km_driven,fuel,seller_type,transmission,owner
3294,13.478865,13.430921,50000,13.088541,12.609423,13.759065,12.964161
2290,12.111783,11.901925,70000,12.457036,12.620399,13.777324,12.962823
874,12.298593,13.336935,50000,12.455670,12.616451,12.645775,12.984195
1907,12.477658,13.055335,92198,12.463101,13.152012,12.637086,12.452642
3244,12.397144,12.857059,3240,12.454647,12.614863,12.627399,12.471970


Обучите несколько деревьев, перебирая максимальную глубину алгоритма из списка `max_depth_list`, а остальные параметры оставьте дефолтными. Выведите лучшее значение гиперпараметра. Постройте график зависимости MSLE на тестовой выборке от значения гиперпараметра. Воспользуйтесь `Pipeline` без `GridSearch`. Проделайте то же самое с `min_samples_split`, `min_impurity_decrease`, `max_leaf_nodes`. (по 2б на каждый параметр)

In [95]:
from sklearn.metrics import mean_squared_error as mse
from sklearn.tree import DecisionTreeRegressor
from sklearn.pipeline import Pipeline
np.random.seed(1)
max_depth_list = [1, 2, 3, 5, 8, 12]

for col in max_depth_list:
### Your code is here
    pipe = Pipeline([ ("custom_transformer", 
                      MeanTargetEncoderNoise(categorical=object_cols, numeric=num_cols) ),
                     ('decision_tree', 
                     DecisionTreeRegressor(max_depth=col))
    ])

    pipe.fit(X_train, y_train)

    train_preds = pipe.predict(X_train)
    test_preds = pipe.predict(X_test)

    train_error = np.mean((train_preds - y_train)**2)
    test_error = np.mean((test_preds - y_test)**2)


    #print(f"Качество на трейне: {col= } {train_error.round(3)}")
    print(f"max_depth: {col} Качество на тесте: {test_error.round(3)}")

max_depth: 1 Качество на тесте: 0.521
max_depth: 2 Качество на тесте: 0.627
max_depth: 3 Качество на тесте: 0.786
max_depth: 5 Качество на тесте: 1.441
max_depth: 8 Качество на тесте: 1.522
max_depth: 12 Качество на тесте: 1.477


In [96]:
min_samples_split_list = [10, 50, 100, 500]

np.random.seed(1)

for col in min_samples_split_list:
### Your code is here
    pipe = Pipeline([ ("custom_transformer", 
                      MeanTargetEncoderNoise(categorical=object_cols, numeric=num_cols) ),
                     ('decision_tree', 
                     DecisionTreeRegressor(min_samples_split=col))
    ])

    pipe.fit(X_train, y_train)

    train_preds = pipe.predict(X_train)
    test_preds = pipe.predict(X_test)

    train_error = np.mean((train_preds - y_train)**2)
    test_error = np.mean((test_preds - y_test)**2)


    #print(f"Качество на трейне: {col= } {train_error.round(3)}")
    print(f"min_samples_split: {col} Качество на тесте: {test_error.round(3)}")


min_samples_split: 10 Качество на тесте: 2.009
min_samples_split: 50 Качество на тесте: 1.387
min_samples_split: 100 Качество на тесте: 0.992
min_samples_split: 500 Качество на тесте: 0.814


In [97]:
min_impurity_decrease_list = [0, 0.1, 0.15, 0.2]

np.random.seed(1)

for col in min_impurity_decrease_list:
### Your code is here
    pipe = Pipeline([ ("custom_transformer", 
                      MeanTargetEncoderNoise(categorical=object_cols, numeric=num_cols) ),
                     ('decision_tree', 
                     DecisionTreeRegressor(min_impurity_decrease=col))
    ])

    pipe.fit(X_train, y_train)

    train_preds = pipe.predict(X_train)
    test_preds = pipe.predict(X_test)

    train_error = np.mean((train_preds - y_train)**2)
    test_error = np.mean((test_preds - y_test)**2)


    #print(f"Качество на трейне: {col= } {train_error.round(3)}")
    print(f"min_impurity_decrease: {col} Качество на тесте: {test_error.round(3)}")

min_impurity_decrease: 0 Качество на тесте: 2.022
min_impurity_decrease: 0.1 Качество на тесте: 0.523
min_impurity_decrease: 0.15 Качество на тесте: 0.521
min_impurity_decrease: 0.2 Качество на тесте: 0.523


In [98]:
max_leaf_nodes_list = [100, 200, 500]

np.random.seed(1)

for col in max_leaf_nodes_list:
### Your code is here
    pipe = Pipeline([ ("custom_transformer", 
                      MeanTargetEncoderNoise(categorical=object_cols, numeric=num_cols) ),
                     ('decision_tree', 
                     DecisionTreeRegressor(max_leaf_nodes=col))
    ])

    pipe.fit(X_train, y_train)

    train_preds = pipe.predict(X_train)
    test_preds = pipe.predict(X_test)

    train_error = np.mean((train_preds - y_train)**2)
    test_error = np.mean((test_preds - y_test)**2)


    #print(f"Качество на трейне: {col= } {train_error.round(3)}")
    print(f"max_leaf_nodes: {col} Качество на тесте: {test_error.round(3)}")

max_leaf_nodes: 100 Качество на тесте: 1.233
max_leaf_nodes: 200 Качество на тесте: 1.989
max_leaf_nodes: 500 Качество на тесте: 1.583


In [99]:
### применим лучшие результаты

pipe = Pipeline([ ("custom_transformer", 
                  MeanTargetEncoderNoise(categorical=object_cols, numeric=num_cols) ),
                 ('decision_tree', 
                 DecisionTreeRegressor(max_depth=3, min_samples_split=500, min_impurity_decrease=0.15, max_leaf_nodes=100))
])

pipe.fit(X_train, y_train)

train_preds = pipe.predict(X_train)
test_preds = pipe.predict(X_test)

train_error = np.mean((train_preds - y_train)**2)
test_error = np.mean((test_preds - y_test)**2)

print(f"Качество на тесте: {test_error.round(3)}")

Качество на тесте: 0.523


Подберите лучшую комбинацию параметров, используя `GridSearchCV` и набор массивов значений параметров из предыдущего задания. Для лучшей комбинации посчитайте MSLE на тестовой выборке. Получились ли лучшие параметры такими же, как если бы вы подбирали их по-отдельности при остальных гиперпараметрах по умолчанию (предыдущее задание)? (2б)

In [100]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    "decision_tree__max_depth": [3, 5, 8, 12],
    "decision_tree__min_samples_split": [10, 50, 100, 500],
    "decision_tree__min_impurity_decrease": [0, 0.1, 0.15, 0.2],
    "decision_tree__max_leaf_nodes": [100, 200, 500]
}
np.random.seed(1)

### Your code is here
from sklearn.model_selection import TimeSeriesSplit
splitter = TimeSeriesSplit(n_splits=3)
search = GridSearchCV(pipe, 
                      param_grid, 
                      cv=splitter,
                      scoring='neg_mean_squared_error',
                      )

search.fit(X_train, y_train)

print(f"Best parameter (CV score={search.best_score_:.5f}):")
print(search.best_params_)

Best parameter (CV score=-0.66726):
{'decision_tree__max_depth': 8, 'decision_tree__max_leaf_nodes': 500, 'decision_tree__min_impurity_decrease': 0.2, 'decision_tree__min_samples_split': 500}


In [101]:
print(f"Качество лучшей модели на финальном тесте: {-search.score(X_test, y_test):.2f}")

Качество лучшей модели на финальном тесте: 0.52
